In [ ]:
import json

In [ ]:
dbutils.widgets.text(
    "config_path",
    ""
)

In [ ]:
config_path = dbutils.widgets.get("config_path")

In [ ]:
config_df = (
    spark.read
         .option("header", True)
         .csv(config_path)
)

In [ ]:
print(f"Config loaded from path: {config_path}")
print(f"Total rows: {config_df.count()}")

In [ ]:

#Build Config JSON & Publish


# Build config list 
table_configs = [
    {
        "data_source": row["data_source"],
        "report_type": row["validation"],
        "secret_scope": row["scope_name"],

        "source_catalog": row["source_catalog"],
        "source_schema": row["source_schema"],
        "source_table": row["source_table"],

        "target_catalog": row["target_catalog"],
        "target_schema": row["target_schema"],
        "target_table": row["target_table_name"],

        "join_columns": row["primary_key"].split(",") if row["primary_key"] else [],

        
        "filters_column": row["filters_column"],
        "filters_condition": row["filters_condition"]
    }
    for row in config_df.collect()
]

# Publish JSON for downstream runner task
dbutils.jobs.taskValues.set(
    key="result",
    value=json.dumps(table_configs)
)

print("Config published")